## ResNet

In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPooling2D, AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from utils import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

Using TensorFlow backend.


In [2]:
epochs = 12
batch_size = 100
data_augmentation = False
img_size = 28

num_classes = 10
num_filters = 128
num_blocks = 4
num_sub_blocks = 2
use_max_pool = False

In [3]:
X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])
X_train = X_train/255
X_test = X_test/255
print(X_test.shape)
print(X_train.shape)
print(y_test.shape)
print(y_train.shape)

(10000, 28, 28, 1)
(60000, 28, 28, 1)
(10000,)
(60000,)


In [4]:
input_size = (img_size, img_size,1)
#Creating model based on ResNet published archietecture
inputs = Input(shape=input_size)
x = Conv2D(num_filters, padding='same', 
           kernel_initializer='he_normal', 
           kernel_size=7, strides=2,
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#Check by applying max pooling later (setting it false as size of image is small i.e. 28x28)
if use_max_pool:
    x = MaxPooling2D(pool_size=3,padding='same', strides=2)(x)
    num_blocks =3
#Creating Conv base stack 

# Instantiate convolutional base (stack of blocks).
for i in range(num_blocks):
    for j in range(num_sub_blocks):
        strides = 1
        is_first_layer_but_not_first_block = j == 0 and i > 0
        if is_first_layer_but_not_first_block:
            strides = 2
        #Creating residual mapping using y
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   strides=strides,
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if is_first_layer_but_not_first_block:
            x = Conv2D(num_filters,
                       kernel_size=1,
                       padding='same',
                       strides=2,
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(x)
        #Adding back residual mapping
        x = keras.layers.add([x, y])
        x = Activation('relu')(x)

    num_filters = 2 * num_filters

# Add classifier on top.
x = AveragePooling2D()(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                activation='softmax',
                kernel_initializer='he_normal')(y)

# Instantiate and compile model.
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 128)  6400        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 14, 14, 128)  512         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 14, 14, 128)  0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [5]:
save_dir = os.path.join(os.getcwd(), 'saved_model')
model_name = 'fmnist_resnet_model.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir,model_name)
print(filepath)

E:\MD\3rd semester\SPR\project\fashion-mnist-master\fashion-mnist-master\saved_model\fmnist_resnet_model.h5


In [6]:
# Prepare callbacks for model saving and for learning rate decaying.
checkpoint = ModelCheckpoint(filepath=filepath,
                             verbose=1,
                             save_best_only=True)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [checkpoint, lr_reducer]

In [7]:
Y_train = keras.utils.np_utils.to_categorical(y_train)
Y_test = keras.utils.np_utils.to_categorical(y_test)
print(Y_test.shape)

(10000, 10)


In [8]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, Y_test),
              shuffle=True,
              callbacks=callbacks)

Not using data augmentation.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 129s 2ms/step - loss: 1.8573 - accuracy: 0.8498 - val_loss: 1.2564 - val_accuracy: 0.8805

Epoch 00001: val_loss improved from inf to 1.25637, saving model to E:\MD\3rd semester\SPR\project\fashion-mnist-master\fashion-mnist-master\saved_model\fmnist_resnet_model.h5
Epoch 2/12
60000/60000 [==============================] - 127s 2ms/step - loss: 0.9809 - accuracy: 0.8957 - val_loss: 0.8139 - val_accuracy: 0.8926

Epoch 00002: val_loss improved from 1.25637 to 0.81393, saving model to E:\MD\3rd semester\SPR\project\fashion-mnist-master\fashion-mnist-master\saved_model\fmnist_resnet_model.h5
Epoch 3/12
60000/60000 [==============================] - 128s 2ms/step - loss: 0.6823 - accuracy: 0.9098 - val_loss: 0.6400 - val_accuracy: 0.8989

Epoch 00003: val_loss improved from 0.81393 to 0.64002, saving model to E:\MD\3rd semester\SPR\project\fashion-mnist-m

In [9]:
scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 10s 1ms/step
Test loss: 0.49007431011199953
Test accuracy: 0.9136000275611877


#### With 12 epoch it gace 91.3% accuracy, with 100 it gave close to 92% accuracy